In [2]:
# === NumPy, SciPy e otimização ===
import numpy as np
from scipy.optimize import minimize
  
# === Visualização ===
import matplotlib.pyplot as plt
from qiskit.visualization import plot_histogram, array_to_latex, plot_state_city
from rustworkx.visualization import mpl_draw as draw_graph
import time

# === Estruturas de grafos ===
import networkx as nx
import rustworkx as rx

# === Qiskit: circuitos, operadores e transpiler ===
from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import SparsePauliOp, Statevector, Operator
from qiskit.circuit.library import QAOAAnsatz, PauliEvolutionGate
from qiskit.synthesis.evolution import LieTrotter
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.circuit import QuantumCircuit, Parameter, ParameterVector

# === Qiskit Aer (simulação local) ===
from qiskit_aer import AerSimulator
from qiskit_aer.primitives import Estimator as EstimatorV2, Sampler as AerSampler

# === Qiskit Runtime (execução em hardware IBM) ===
from qiskit_ibm_runtime import QiskitRuntimeService, Session, EstimatorV2, SamplerV2

# === Qiskit Primitives ===
# from qiskit.primitives import Sampler, BaseSamplerV2

# === Qiskit Optimization === 
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit_optimization.minimum_eigensolvers import QAOA
from qiskit_optimization.optimizers import COBYLA, NELDER_MEAD

# ---- Importações necessárias (atualizadas para Qiskit 2.x) ----
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit_aer import AerSimulator
from qiskit.primitives import StatevectorEstimator as Estimator  # Correção: Use StatevectorEstimator
from qiskit.primitives import StatevectorSampler as Sampler  # Correção: Use StatevectorSampler
from qiskit.circuit.library import QAOAAnsatz  # Novo: QAOAAnsatz em vez de QAOA
from qiskit.quantum_info import SparsePauliOp  # Para operadores
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager  # transpiler para versões antigas/compatíveis
from qiskit import transpile
import numpy as np
from scipy.optimize import minimize  # Otimizador clássico externo 

from qiskit.providers.basic_provider import BasicSimulator

In [ ]:


# '''
# Métodos para execução da estratégia QAOA
# '''

# # Conecção com a IBM

# token_privado = " c06b01c23c344e2abd796c04f20858e0" # <-- Inserir token privado na IBM
# instance_do_projeto = "" # <-- Inserir instance do projeto

# from qiskit_ibm_runtime import QiskitRuntimeService
# service = QiskitRuntimeService(
#     channel="ibm_cloud",
#     token = token_privado,
#     instance = instance_do_projeto
#     )

# ## Backend fake
# from qiskit_ibm_runtime.fake_provider import FakeBrisbane
# backend = FakeBrisbane() # FakeBrisbane(): 127 qubits. Mais fake backends: 'https://quantum.cloud.ibm.com/docs/en/api/qiskit-ibm-runtime/fake-provider'

# # Importação do Estimator
# from qiskit_aer.primitives import EstimatorV2
# Estimator = EstimatorV2

# # Importação do Sampler
# ## Simulação local
# from qiskit.primitives import StatevectorSampler # For local simulation
# sampler = StatevectorSampler()
# ## Simulação na IBM:
# # from qiskit_ibm_runtime import SamplerV2
# # sampler = SamplerV2(mode = backend)

# # Transpilador

# from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
# pass_manager = generate_preset_pass_manager(optimization_level=1, backend=backend)

In [ ]:

# Dados do problema

valor_itens = [12,2,1,1,4]
peso_itens = [4,2,2,1,10]
capacidade = 15
qtd_itens = len(valor_itens)
penalidade = 100

print(f"Capacidade: {capacidade}")
print(f"Peso dos itens: {peso_itens}, Quantidade de itens: {len(valor_itens)}")
print(f"Valor dos itens: {valor_itens}, Quantidade de itens: {len(peso_itens)}")

# Função para criar a matriz QUBO (sem alterações, está correta)
def cria_qubo(valor_itens, peso_itens, capacidade, penalidade): 
    n = len(valor_itens)  
    qubo_matrix = np.zeros([n, n]) 
    for i in range(n):
        qubo_matrix[i, i] = -valor_itens[i] - penalidade * (peso_itens[i]**2)
        for j in range(i + 1, n):
            qubo_matrix[i, j] = 2 * penalidade * peso_itens[i] * peso_itens[j]
    for i in range(n):
        qubo_matrix[i, i] += 2 * penalidade * peso_itens[i] * capacidade
    return qubo_matrix 

qubo_matrix = cria_qubo(valor_itens, peso_itens, capacidade, penalidade) 
print("Matriz QUBO:")
print(qubo_matrix)

# Função para criar o Hamiltoniano de custo (corrigida: sinal positivo para quadráticos, loop i < j)
def hamiltanino_de_custo(qubo_matrix):
    n = len(qubo_matrix)
    h_custo = SparsePauliOp("I" * n, coeffs=[0])
    
    # Termos lineares (diagonais)
    for i in range(n):
        pauli_str_diag = ["I"] * n
        pauli_str_diag[i] = "Z"
        coef_diag = -qubo_matrix[i, i] / 2  # -q_{i,i}/2 para Z_i
        h_custo += SparsePauliOp("".join(pauli_str_diag), coeffs=[coef_diag])
    
    # Termos quadráticos (i < j)
    for i in range(n):
        for j in range(i + 1, n):  # Apenas i < j para evitar repetição
            if qubo_matrix[i, j] != 0:
                pauli_str_quad = ["I"] * n
                pauli_str_quad[i] = "Z"
                pauli_str_quad[j] = "Z"
                coef_quad = qubo_matrix[i, j] / 4  # +q_{i,j}/4 para Z_i Z_j
                h_custo += SparsePauliOp("".join(pauli_str_quad), coeffs=[coef_quad])
    
    return h_custo

hamiltoniano_c = hamiltanino_de_custo(qubo_matrix)
print("Hamiltoniano de custo:")
print(hamiltoniano_c)

# Função para criar o Hamiltoniano de mistura (simplificado para QAOA puro, com opção de mixer personalizado)
def hamiltanino_de_mixer(valor_itens, peso_itens):
    n = len(valor_itens)
    h_mix = SparsePauliOp("I" * n, coeffs=[0])
    
    # Mixer personalizado: sum (v_i/w_i)_norm X_i
    razao = [valor_itens[i] / peso_itens[i] for i in range(n)]
    max_razao = max(razao)
    normaliza = [r / max_razao for r in razao]  # Normaliza v_i/w_i
    
    for i in range(n):
        pauli_str_mix = ["I"] * n
        pauli_str_mix[i] = "X"
        h_mix += SparsePauliOp("".join(pauli_str_mix), coeffs=[normaliza[i]])
    
    return h_mix

hamiltoniano_m = hamiltanino_de_mixer(valor_itens, peso_itens)
print("Hamiltoniano de mistura:")
print(hamiltoniano_m)



# Função para avaliar bitstrings
def avalia_bitstring(bitstring, valor_itens, peso_itens, capacidade):
    bits = [int(b) for b in bitstring[::-1]]  # Inverte (ordem Qiskit)
    valor_total = sum(valor_itens[i] * bits[i] for i in range(len(bits)))
    peso_total = sum(peso_itens[i] * bits[i] for i in range(len(bits)))
    valido = peso_total <= capacidade
    return valor_total, peso_total, valido

# Função para calcular o custo esperado (usado na otimização)
def custo_esperado(params, qc, gama, beta, profundidade, backend, shots=1000):
    params_dict = {gama[i]: params[i] for i in range(profundidade)}
    params_dict.update({beta[i]: params[profundidade + i] for i in range(profundidade)})
    assigned_qc = qc.assign_parameters(params_dict)
    transpiled_qc = transpile(assigned_qc, backend)
    result = backend.run(transpiled_qc, shots=shots).result()
    counts = result.get_counts()
    
    custo_total = 0
    for bitstring, freq in counts.items():
        bits = [int(b) for b in bitstring[::-1]]
        # Calcule o custo do QUBO: -sum(v_i x_i) + lambda (sum(w_i x_i) - W)^2
        valor = sum(valor_itens[i] * bits[i] for i in range(len(bits)))
        peso = sum(peso_itens[i] * bits[i] for i in range(len(bits)))
        penalidade_term = penalidade * (peso - capacidade) ** 2
        custo = -valor + penalidade_term
        custo_total += custo * (freq / shots)
    
    return custo_total

# Otimizar parâmetros gama e beta
backend = AerSimulator()
shots = 10000  # Aumentado para mais precisão
initial_params = np.random.uniform(0, np.pi, 2 * profundidade)
initial_params = np.random.uniform(0, np.pi, 2 * profundidade)  # [γ1, γ2, β1, β2]
result = minimize(
    fun=custo_esperado,
    x0=initial_params,
    args=(qc, gama, beta, profundidade, backend, shots),
    method="COBYLA",
    options={"maxiter": 100}
)

# Melhores parâmetros
optimal_params = result.x
print("Parâmetros otimizados (γ, β):", optimal_params)

# Executar com parâmetros otimizados
params_dict = {gama[i]: optimal_params[i] for i in range(profundidade)}
params_dict.update({beta[i]: optimal_params[profundidade + i] for i in range(profundidade)})
assigned_qc = qc.assign_parameters(params_dict)
transpiled_qc = transpile(assigned_qc, backend)
job = backend.run(transpiled_qc, shots=shots)
result = job.result()
counts = result.get_counts()
print("Contagens de bitstrings (otimizadas):")
print(counts)

# Visualizar distribuição
plot_histogram(counts)

# Pós-processar resultados
top_results = sorted(counts.items(), key=lambda x: x[1], reverse=True)[:5]
print("Top 5 soluções (otimizadas):")
best_valor = -1
best_bitstring = None
for bitstring, freq in top_results:
    valor, peso, valido = avalia_bitstring(bitstring, valor_itens, peso_itens, capacidade)
    print(f"Bitstring: {bitstring}, Frequência: {freq}, Valor: {valor}, Peso: {peso}, Válido: {valido}")
    if valido and valor > best_valor:
        best_valor = valor
        best_bitstring = bitstring # type: ignore

    # best_bitstring_reverse = best_bitstring[::1]


print(f"\nMelhor solução válida: Bitstring {best_bitstring}, Valor {best_valor}")



2.96932352 4.11774822 2.2534057  0.05242743 1.0043802  1.56648727
 1.6600594  2.30763525

In [ ]:
# Função para criar o circuito QAOA (sem alterações, está correta)
def cria_qaoa(hamiltoniano_c, hamiltoniano_m, profundidade): 
    n = hamiltoniano_c.num_qubits
    qc = QuantumCircuit(n)
    
    # Inicialização
    qc.h(range(n))
    
    # Parâmetros
    gama = ParameterVector("γ", profundidade)
    beta = ParameterVector("β", profundidade)
    
    # Camadas QAOA
    for i in range(profundidade):
        evo_custo = PauliEvolutionGate(hamiltoniano_c, time=gama[i])
        qc.append(evo_custo, range(n))
        evo_mixer = PauliEvolutionGate(hamiltoniano_m, time=beta[i])
        qc.append(evo_mixer, range(n))
    
    # Medições
    qc.measure_all()
    
    return qc, gama, beta

profundidade = 1
qc, gama, beta = cria_qaoa(hamiltoniano_c, hamiltoniano_m, profundidade)
print("Circuito QAOA criado com sucesso!")
qc.draw("mpl")  # Comente para evitar abrir a janela gráfica

In [ ]:
plot_histogram(counts)


In [ ]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import PauliEvolutionGate
from qiskit_aer import AerSimulator
from qiskit import transpile
from qiskit.visualization import plot_histogram
from scipy.optimize import minimize

# Dados do problema
# valor_itens = [8, 47, 10, 5, 16]
# peso_itens = [3, 11, 14, 19, 5]
# capacidade = 26
# qtd_itens = len(valor_itens)
# penalidade = 100

# valor_itens = [12,2,1,1,4]
# peso_itens = [4,2,2,1,10]
# capacidade = 15
# qtd_itens = len(valor_itens)
# penalidade = 100

valor_itens = [220, 208, 198, 192, 180, 180, 165, 162, 160, 158, 155, 130, 125, 122, 120, 118, 115, 110, 105, 101]
peso_itens = [80, 82, 85, 70, 72, 70, 66, 50, 55, 25, 50, 55, 40, 48, 59, 32, 22, 60, 30, 32]
capacidade = 500
qtd_itens = len(valor_itens)
penalidade = 500

print(f"Capacidade: {capacidade}")
print(f"Peso dos itens: {peso_itens}, Quantidade de itens: {len(valor_itens)}")
print(f"Valor dos itens: {valor_itens}, Quantidade de itens: {len(peso_itens)}")

print(f"Capacidade: {capacidade}")
print(f"Peso dos itens: {peso_itens}, Quantidade de itens: {len(valor_itens)}")
print(f"Valor dos itens: {valor_itens}, Quantidade de itens: {len(peso_itens)}")

# Função para criar a matriz QUBO (sem alterações, está correta)
def cria_qubo(valor_itens, peso_itens, capacidade, penalidade): 
    n = len(valor_itens)  
    qubo_matrix = np.zeros([n, n]) 
    for i in range(n):
        qubo_matrix[i, i] = -valor_itens[i] - penalidade * (peso_itens[i]**2)
        for j in range(i + 1, n):
            qubo_matrix[i, j] = 2 * penalidade * peso_itens[i] * peso_itens[j]
    for i in range(n):
        qubo_matrix[i, i] += 2 * penalidade * peso_itens[i] * capacidade
    return qubo_matrix 

qubo_matrix = cria_qubo(valor_itens, peso_itens, capacidade, penalidade) 
print("Matriz QUBO:")
print(qubo_matrix)

# Função para criar o Hamiltoniano de custo (sem alterações, está correta)
def hamiltanino_de_custo(qubo_matrix):
    n = len(qubo_matrix)
    h_custo = SparsePauliOp("I" * n, coeffs=[0])
    
    # Termos lineares (diagonais)
    for i in range(n):
        pauli_str_diag = ["I"] * n
        pauli_str_diag[i] = "Z"
        coef_diag = -qubo_matrix[i, i] / 2  # -q_{i,i}/2 para Z_i
        h_custo += SparsePauliOp("".join(pauli_str_diag), coeffs=[coef_diag])
    
    # Termos quadráticos (i < j)
    for i in range(n):
        for j in range(i + 1, n):  # Apenas i < j para evitar repetição
            if qubo_matrix[i, j] != 0:
                pauli_str_quad = ["I"] * n
                pauli_str_quad[i] = "Z"
                pauli_str_quad[j] = "Z"
                coef_quad = qubo_matrix[i, j] / 4  # +q_{i,j}/4 para Z_i Z_j
                h_custo += SparsePauliOp("".join(pauli_str_quad), coeffs=[coef_quad])
    
    return h_custo

hamiltoniano_c = hamiltanino_de_custo(qubo_matrix)
print("Hamiltoniano de custo:")
print(hamiltoniano_c)

# Função para criar o Hamiltoniano de mistura (removidos Y_i e densidade)
def hamiltanino_de_mixer(valor_itens, peso_itens):
    n = len(valor_itens)
    h_mix = SparsePauliOp("I" * n, coeffs=[0])
    
    # Mixer personalizado: sum (v_i/w_i)_norm X_i
    razao = [valor_itens[i] / peso_itens[i] for i in range(n)]
    max_razao = max(razao)
    normaliza = [r / max_razao for r in razao]  # Normaliza v_i/w_i
    
    for i in range(n):
        pauli_str_mix = ["I"] * n
        pauli_str_mix[i] = "X"
        h_mix += SparsePauliOp("".join(pauli_str_mix), coeffs=[normaliza[i]])
    
    return h_mix

hamiltoniano_m = hamiltanino_de_mixer(valor_itens, peso_itens)
print("Hamiltoniano de mistura:")
print(hamiltoniano_m)

# Função para criar o circuito QAOA (sem alterações, está correta)
def cria_qaoa(hamiltoniano_c, hamiltoniano_m, profundidade): 
    n = hamiltoniano_c.num_qubits
    qc = QuantumCircuit(n)
    
    # Inicialização
    qc.h(range(n))
    
    # Parâmetros
    gama = ParameterVector("γ", profundidade)
    beta = ParameterVector("β", profundidade)
    
    # Camadas QAOA
    for i in range(profundidade):
        evo_custo = PauliEvolutionGate(hamiltoniano_c, time=gama[i])
        qc.append(evo_custo, range(n))
        evo_mixer = PauliEvolutionGate(hamiltoniano_m, time=beta[i])
        qc.append(evo_mixer, range(n))
    
    # Medições
    qc.measure_all()
    
    return qc, gama, beta

profundidade = 4
qc, gama, beta = cria_qaoa(hamiltoniano_c, hamiltoniano_m, profundidade)
print("Circuito QAOA criado com sucesso!")
qc.draw("mpl")  # Comente para evitar abrir a janela gráfica

# Função para avaliar bitstrings
def avalia_bitstring(bitstring, valor_itens, peso_itens, capacidade):
    bits = [int(b) for b in bitstring[::-1]]  # Inverte (ordem Qiskit)
    valor_total = sum(valor_itens[i] * bits[i] for i in range(len(bits)))
    peso_total = sum(peso_itens[i] * bits[i] for i in range(len(bits)))
    valido = peso_total <= capacidade
    return valor_total, peso_total, valido

# Função para calcular o custo esperado (corrigida para usar qubo_matrix)
def custo_esperado(params, qc, gama, beta, profundidade, backend, shots=1000):
    params_dict = {gama[i]: params[i] for i in range(profundidade)}
    params_dict.update({beta[i]: params[profundidade + i] for i in range(profundidade)})
    assigned_qc = qc.assign_parameters(params_dict)
    transpiled_qc = transpile(assigned_qc, backend)
    result = backend.run(transpiled_qc, shots=shots).result()
    counts = result.get_counts()
    
    custo_total = 0
    for bitstring, freq in counts.items():
        bits = [int(b) for b in bitstring[::-1]]
        custo = sum(qubo_matrix[i, j] * bits[i] * bits[j] for i in range(len(bits)) for j in range(i, len(bits)))
        custo_total += custo * (freq / shots)
    
    return custo_total

# Otimizar parâmetros gama e beta
backend = AerSimulator()
shots = 10000  # Aumentado para mais precisão
initial_params = np.random.uniform(0, np.pi, 2 * profundidade)  # [γ1, γ2, γ3, γ4, β1, β2, β3, β4]
result = minimize(
    fun=custo_esperado,
    x0=initial_params,
    args=(qc, gama, beta, profundidade, backend, shots),
    method="COBYLA",
    options={"maxiter": 200}  # Aumentado para mais iterações
)

# Melhores parâmetros
optimal_params = result.x
print("Parâmetros otimizados (γ, β):", optimal_params)

# Executar com parâmetros otimizados
params_dict = {gama[i]: optimal_params[i] for i in range(profundidade)}
params_dict.update({beta[i]: optimal_params[profundidade + i] for i in range(profundidade)})
assigned_qc = qc.assign_parameters(params_dict)
transpiled_qc = transpile(assigned_qc, backend)
job = backend.run(transpiled_qc, shots=shots)
result = job.result()
counts = result.get_counts()
print("Contagens de bitstrings (otimizadas):")
print(counts)

# Visualizar distribuição
plot_histogram(counts)

# Pós-processar resultados (corrigido para maior robustez)
top_results = sorted(counts.items(), key=lambda x: x[1], reverse=True)[:5]
print("Top 5 soluções (otimizadas):")
best_valor = -float('inf')  # Corrigido para evitar erros com valores negativos
best_bitstring = None
for bitstring, freq in top_results:
    valor, peso, valido = avalia_bitstring(bitstring, valor_itens, peso_itens, capacidade)
    print(f"Bitstring: {bitstring}, Frequência: {freq}, Valor: {valor}, Peso: {peso}, Válido: {valido}")
    if valido and valor > best_valor:
        best_valor = valor
        best_bitstring = bitstring

if best_bitstring is None:
    print("Nenhuma solução válida encontrada.")
else:
    print(f"\nMelhor solução válida: Bitstring {best_bitstring}, Valor {best_valor}")

In [ ]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import PauliEvolutionGate
from qiskit.providers.basic_provider import BasicSimulator  # Novo backend
from qiskit import transpile
from qiskit.visualization import plot_histogram
from scipy.optimize import minimize

# Dados do problema
# valor_itens = [8, 47, 10, 5, 16]
# peso_itens = [3, 11, 14, 19, 5]
# capacidade = 26
# qtd_itens = len(valor_itens)
# penalidade = 100


# valor_itens = [12,2,1,1,4]
# peso_itens = [4,2,2,1,10]
# capacidade = 15
# qtd_itens = len(valor_itens)
# penalidade = 100


# print(f"Capacidade: {capacidade}")
# print(f"Peso dos itens: {peso_itens}, Quantidade de itens: {qtd_itens}")
# print(f"Valor dos itens: {valor_itens}, Quantidade de itens: {qtd_itens}")


# valor_itens = [80, 82, 85, 70, 72, 70, 66, 50, 55, 25, 50, 55, 40, 48, 59, 32, 22,60, 30, 32, 40, 38, 35, 32, 25, 28, 30, 22, 50, 30, 45, 30, 60, 50, 20,65, 20, 25, 30, 10, 20, 25, 15, 10, 10, 10, 4, 4, 2, 1]
# peso_itens = [220, 208, 198, 192, 180, 180, 165, 162, 160, 158, 155, 130, 125,122, 120, 118, 115, 110, 105, 101, 100, 100, 98, 96, 95, 90, 88, 82,80, 77, 75, 73, 72, 70, 69, 66, 65, 63, 60, 58, 56, 50, 30, 20, 15, 10, 8,5, 3, 1]
# capacidade = 1000
# qtd_itens = len(valor_itens)
# penalidade = 100
# print(f"Capacidade: {capacidade}")
# print(f"Peso dos itens: {peso_itens}, Quantidade de itens: {qtd_itens}")
# print(f"Valor dos itens: {valor_itens}, Quantidade de itens: {qtd_itens}")


valor_itens = [220, 208, 198, 192, 180, 180, 165, 162, 160, 158, 155, 130, 125, 122, 120, 118, 115, 110, 105, 101]
peso_itens = [80, 82, 85, 70, 72, 70, 66, 50, 55, 25, 50, 55, 40, 48, 59, 32, 22, 60, 30, 32]
capacidade = 500
qtd_itens = len(valor_itens)
penalidade = 500

print(f"Capacidade: {capacidade}")
print(f"Peso dos itens: {peso_itens}, Quantidade de itens: {len(valor_itens)}")
print(f"Valor dos itens: {valor_itens}, Quantidade de itens: {len(peso_itens)}")

# Função para criar a matriz QUBO
def cria_qubo(valor_itens, peso_itens, capacidade, penalidade): 
    n = len(valor_itens)  
    qubo_matrix = np.zeros([n, n]) 
    for i in range(n):
        qubo_matrix[i, i] = -valor_itens[i] - penalidade * (peso_itens[i]**2)
        for j in range(i + 1, n):
            qubo_matrix[i, j] = 2 * penalidade * peso_itens[i] * peso_itens[j]
    for i in range(n):
        qubo_matrix[i, i] += 2 * penalidade * peso_itens[i] * capacidade
    return qubo_matrix 

qubo_matrix = cria_qubo(valor_itens, peso_itens, capacidade, penalidade) 
print("Matriz QUBO:")
print(qubo_matrix)

# Função para criar o Hamiltoniano de custo
def hamiltanino_de_custo(qubo_matrix):
    n = len(qubo_matrix)
    h_custo = SparsePauliOp("I" * n, coeffs=[0])
    
    for i in range(n):
        pauli_str_diag = ["I"] * n
        pauli_str_diag[i] = "Z"
        coef_diag = -qubo_matrix[i, i] / 2
        h_custo += SparsePauliOp("".join(pauli_str_diag), coeffs=[coef_diag])
    
    for i in range(n):
        for j in range(i + 1, n):
            if qubo_matrix[i, j] != 0:
                pauli_str_quad = ["I"] * n
                pauli_str_quad[i] = "Z"
                pauli_str_quad[j] = "Z"
                coef_quad = qubo_matrix[i, j] / 4
                h_custo += SparsePauliOp("".join(pauli_str_quad), coeffs=[coef_quad])
    
    return h_custo

hamiltoniano_c = hamiltanino_de_custo(qubo_matrix)
print("Hamiltoniano de custo:")
print(hamiltoniano_c)

# Função para criar o Hamiltoniano de mistura
def hamiltanino_de_mixer(valor_itens, peso_itens):
    n = len(valor_itens)
    h_mix = SparsePauliOp("I" * n, coeffs=[0])
    
    razao = [valor_itens[i] / peso_itens[i] for i in range(n)]
    max_razao = max(razao)
    normaliza = [r / max_razao for r in razao]
    
    for i in range(n):
        pauli_str_mix = ["I"] * n
        pauli_str_mix[i] = "X"
        h_mix += SparsePauliOp("".join(pauli_str_mix), coeffs=[normaliza[i]])
    
    return h_mix

hamiltoniano_m = hamiltanino_de_mixer(valor_itens, peso_itens)
print("Hamiltoniano de mistura:")
print(hamiltoniano_m)

# Função para criar o circuito QAOA
def cria_qaoa(hamiltoniano_c, hamiltoniano_m, profundidade): 
    n = hamiltoniano_c.num_qubits
    qc = QuantumCircuit(n)
    
    qc.h(range(n))
    
    gama = ParameterVector("γ", profundidade)
    beta = ParameterVector("β", profundidade)
    
    for i in range(profundidade):
        evo_custo = PauliEvolutionGate(hamiltoniano_c, time=gama[i])
        qc.append(evo_custo, range(n))
        evo_mixer = PauliEvolutionGate(hamiltoniano_m, time=beta[i])
        qc.append(evo_mixer, range(n))
    
    qc.measure_all()
    
    return qc, gama, beta

profundidade = 4
qc, gama, beta = cria_qaoa(hamiltoniano_c, hamiltoniano_m, profundidade)
print("Circuito QAOA criado com sucesso!")
qc.draw("mpl")

# Função para avaliar bitstrings
def avalia_bitstring(bitstring, valor_itens, peso_itens, capacidade):
    bits = [int(b) for b in bitstring[::-1]]  # Inverte para ordem Qiskit
    valor_total = sum(valor_itens[i] * bits[i] for i in range(len(bits)))
    peso_total = sum(peso_itens[i] * bits[i] for i in range(len(bits)))
    valido = peso_total <= capacidade
    return valor_total, peso_total, valido

# Função para calcular o custo esperado
def custo_esperado(params, qc, gama, beta, profundidade, backend, shots=1000):
    params_dict = {gama[i]: params[i] for i in range(profundidade)}
    params_dict.update({beta[i]: params[profundidade + i] for i in range(profundidade)})
    assigned_qc = qc.assign_parameters(params_dict)
    transpiled_qc = transpile(assigned_qc, backend, coupling_map=None)  # Desativa coupling_map
    result = backend.run(transpiled_qc, shots=shots).result()
    counts = result.get_counts()
    
    custo_total = 0
    for bitstring, freq in counts.items():
        bits = [int(b) for b in bitstring[::-1]]
        custo = sum(qubo_matrix[i, j] * bits[i] * bits[j] for i in range(len(bits)) for j in range(i, len(bits)))
        custo_total += custo * (freq / shots)
    
    return custo_total

# Configurar o backend com BasicSimulator
backend = BasicSimulator()
shots = 10000
initial_params = np.random.uniform(0, np.pi, 2 * profundidade)
result = minimize(
    fun=custo_esperado,
    x0=initial_params,
    args=(qc, gama, beta, profundidade, backend, shots),
    method="COBYLA",
    options={"maxiter": 200}
)

# Melhores parâmetros
optimal_params = result.x
print("Parâmetros otimizados (γ, β):", optimal_params)

# Executar com parâmetros otimizados
params_dict = {gama[i]: optimal_params[i] for i in range(profundidade)}
params_dict.update({beta[i]: optimal_params[profundidade + i] for i in range(profundidade)})
assigned_qc = qc.assign_parameters(params_dict)
transpiled_qc = transpile(assigned_qc, backend, coupling_map=None)
job = backend.run(transpiled_qc, shots=shots)
result = job.result()
counts = result.get_counts()
print("Contagens de bitstrings (otimizadas):")
print(counts)

# Visualizar distribuição
plot_histogram(counts)

# Pós-processar resultados
top_results = sorted(counts.items(), key=lambda x: x[1], reverse=True)[:10]
print("Top 10 soluções (otimizadas):")
best_valor = -float('inf')
best_bitstring = None
for bitstring, freq in top_results:
    valor, peso, valido = avalia_bitstring(bitstring, valor_itens, peso_itens, capacidade)
    print(f"Bitstring: {bitstring}, Frequência: {freq}, Valor: {valor}, Peso: {peso}, Válido: {valido}")
    if valido and valor > best_valor:
        best_valor = valor
        best_bitstring = bitstring

if best_bitstring is None:
    print("Nenhuma solução válida encontrada.")
else:
    print(f"\nMelhor solução válida: Bitstring {best_bitstring}, Valor {best_valor}")


In [ ]:
def knapsack_dp(values, weights, capacity):
    n = len(values)
    dp = [[0 for _ in range(capacity + 1)] for _ in range(n + 1)]
    for i in range(1, n + 1):
        for w in range(capacity + 1):
            if weights[i-1] <= w:
                dp[i][w] = max(dp[i-1][w], dp[i-1][w - weights[i-1]] + values[i-1])
            else:
                dp[i][w] = dp[i-1][w]
    return dp[n][capacity]

valor_otimo = knapsack_dp(valor_itens, peso_itens, capacidade)

In [ ]:
print(f"Valor ótimo (programação dinâmica): {valor_otimo}")
print(f"Razão de aproximação: {best_valor / valor_otimo:.4f}" if best_valor != -float('inf') else "Nenhuma solução válida.")

In [ ]:
valor_itens = [95, 4, 60, 32, 23, 72, 80, 62, 65, 46]
peso_itens = [55, 10, 47, 5, 4, 50, 8, 61, 85, 87]
capacidade =  269
qtd_itens = len(valor_itens)
penalidade = max(valor_itens) * 2

print(f"Valor iten: {valor_itens} = Quantidade {qtd_itens}")
print(f"Valor itens {peso_itens} = Quantidade {qtd_itens}")

In [ ]:
# Função para criar a matriz QUBO
def cria_qubo(valor_itens, peso_itens, capacidade, penalidade): 
    n = len(valor_itens)  
    qubo_matrix = np.zeros([n, n]) 
    for i in range(n):
        qubo_matrix[i, i] = -valor_itens[i] + penalidade * (peso_itens[i]**2) - 2 * penalidade * capacidade * peso_itens[i]
        for j in range(i + 1, n):
            qubo_matrix[i, j] = 2 * penalidade * peso_itens[i] * peso_itens[j]
    for i in range(n):
        qubo_matrix[i, i] += 2 * penalidade * peso_itens[i] * capacidade
    return qubo_matrix 

qubo_matrix = cria_qubo(valor_itens, peso_itens, capacidade, penalidade) 
print("Matriz QUBO:")
print(qubo_matrix)

In [ ]:
def hamiltoniano_de_custo_corrigido(qubo_matrix):
    """
    Converte matriz QUBO para Hamiltoniano Ising corretamente
    Relação: x_i = (1 - Z_i)/2
    QUBO: sum_{i,j} Q_{ij} x_i x_j
    Ising: constant + sum_i h_i Z_i + sum_{i<j} J_{ij} Z_i Z_j
    """
    n = len(qubo_matrix)
    constant_term = 0
    h_custo = SparsePauliOp("I" * n, coeffs=[0])
    
    # Termos lineares (Z_i)
    for i in range(n):
        # Coeficiente para Z_i: -Q_ii/2 - sum_{j≠i} Q_ij/4
        coef_linear = -qubo_matrix[i, i] / 2
        
        # Contribuição dos termos off-diagonais
        for j in range(n):
            if j != i:
                coef_linear -= qubo_matrix[i, j] / 4
        
        pauli_str = ["I"] * n
        pauli_str[i] = "Z"
        h_custo += SparsePauliOp("".join(pauli_str), coeffs=[coef_linear])
        
        # Acumular termo constante
        constant_term += qubo_matrix[i, i] / 2
        for j in range(i + 1, n):
            constant_term += qubo_matrix[i, j] / 4
    
    # Termos quadráticos (Z_i Z_j)
    for i in range(n):
        for j in range(i + 1, n):
            if qubo_matrix[i, j] != 0:
                pauli_str = ["I"] * n
                pauli_str[i] = "Z"
                pauli_str[j] = "Z"
                coef_quad = qubo_matrix[i, j] / 4
                h_custo += SparsePauliOp("".join(pauli_str), coeffs=[coef_quad])
    
    # Adicionar termo constante (opcional, mas recomendado)
    h_custo += SparsePauliOp("I" * n, coeffs=[constant_term])
    
    return h_custo

m_h_c = hamiltoniano_de_custo_corrigido(qubo_matrix)

print(f"Hamiltoniano de Custo {m_h_c}")

In [ ]:
def hamiltoniano_mixer_xy(valor_itens, peso_itens):
    """Mixer com termos X e Y como mencionado na sua teoria"""
    n = len(valor_itens)
    
    razoes = [valor_itens[i] / peso_itens[i] for i in range(n)]
    max_razao = max(razoes)
    c_i = [r / max_razao for r in razoes]
    d_i = [0.5 * c for c in c_i]  
    
    h_mix = SparsePauliOp("I" * n, coeffs=[0])
    
    for i in range(n):
        # Termo X
        pauli_x = ["I"] * n
        pauli_x[i] = "X"
        h_mix += SparsePauliOp("".join(pauli_x), coeffs=[c_i[i]])
        
        # Termo Y  
        pauli_y = ["I"] * n
        pauli_y[i] = "Y"
        h_mix += SparsePauliOp("".join(pauli_y), coeffs=[d_i[i]])
    
    return h_mix

m_h_m = hamiltoniano_mixer_xy(valor_itens, peso_itens)

print(f"Hamiltoniano de mistura {m_h_m}")

In [ ]:
def cria_qaoa_tradicional(hamiltoniano_c, hamiltoniano_m, profundidade):
    """Implementação tradicional do QAOA com decomposição explícita"""
    n = hamiltoniano_c.num_qubits
    qc = QuantumCircuit(n)
    
    # Inicialização em superposição
    qc.h(range(n))
    
    # Parâmetros
    gammas = [Parameter(f'γ_{i}') for i in range(profundidade)]
    betas = [Parameter(f'β_{i}') for i in range(profundidade)]
    
    for p in range(profundidade):
        # Camada do Hamiltoniano de custo
        for pauli_term, coeff in hamiltoniano_c.items():
            if pauli_term == 'I'*n:
                continue  # Ignorar identidade
            
            # Implementar e^{-iγ H_C} para cada termo Pauli
            qc = adiciona_evolucao_pauli(qc, pauli_term, gammas[p] * coeff)
        
        # Camada do mixer
        for pauli_term, coeff in hamiltoniano_m.items():
            if pauli_term == 'I'*n:
                continue
            
            qc = adiciona_evolucao_pauli(qc, pauli_term, betas[p] * coeff)
    
    qc.measure_all()
    return qc, gammas, betas

def adiciona_evolucao_pauli(qc, pauli_string, angle):
    """Adiciona a evolução para um termo Pauli específico"""
    n = len(pauli_string)
    qubits = []
    paulis = []
    
    for i, pauli in enumerate(pauli_string):
        if pauli != 'I':
            qubits.append(i)
            paulis.append(pauli)
    
    if not qubits:
        return qc  # Termo identidade
    
    # Aplica portas de base change se necessário
    for i, (qubit, pauli) in enumerate(zip(qubits, paulis)):
        if pauli == 'X':
            qc.h(qubit)
        elif pauli == 'Y':
            qc.rx(np.pi/2, qubit)  # H + S† para Y
    
    # Aplica CNOT ladder e RZ
    for i in range(len(qubits) - 1):
        qc.cx(qubits[i], qubits[i+1])
    
    qc.rz(2 * angle, qubits[-1])
    
    # Reverte CNOT ladder
    for i in range(len(qubits) - 2, -1, -1):
        qc.cx(qubits[i], qubits[i+1])
    
    # Reverte portas de base change
    for i, (qubit, pauli) in enumerate(zip(qubits, paulis)):
        if pauli == 'X':
            qc.h(qubit)
        elif pauli == 'Y':
            qc.rx(-np.pi/2, qubit)
    
    return qc

profundidade = 4

In [ ]:
import numpy as np
from collections import defaultdict

def avalia_bitstring_melhorada(bitstring, valor_itens, peso_itens, capacidade, penalidade=None):
    """
    Avalia bitstring com opção de penalidade para soluções inválidas
    """
    bits = [int(b) for b in bitstring[::-1]]  # Inverte para ordem Qiskit
    n = len(bits)
    
    valor_total = sum(valor_itens[i] * bits[i] for i in range(n))
    peso_total = sum(peso_itens[i] * bits[i] for i in range(n))
    valido = peso_total <= capacidade
    
    # Aplica penalidade se fornecida
    if not valido and penalidade is not None:
        valor_total -= penalidade * (peso_total - capacidade)
    
    return {
        'bitstring': bitstring,
        'valor': valor_total,
        'peso': peso_total,
        'valido': valido,
        'bits': bits
    }

def analisa_resultados_qaoa(counts, valor_itens, peso_itens, capacidade, top_k=10):
    """
    Analisa todos os resultados do QAOA e retorna as melhores soluções
    """
    resultados = []
    
    for bitstring, freq in counts.items():
        resultado = avalia_bitstring_melhorada(bitstring, valor_itens, peso_itens, capacidade)
        resultado['frequencia'] = freq
        resultados.append(resultado)
    
    # Ordena por valor (considerando apenas soluções válidas)
    validos = [r for r in resultados if r['valido']]
    invalidos = [r for r in resultados if not r['valido']]
    
    validos_ordenados = sorted(validos, key=lambda x: x['valor'], reverse=True)
    invalidos_ordenados = sorted(invalidos, key=lambda x: x['valor'], reverse=True)
    
    print("=== MELHORES SOLUÇÕES VÁLIDAS ===")
    for i, resultado in enumerate(validos_ordenados[:top_k]):
        print(f"{i+1}. {resultado['bitstring']} | "
              f"Valor: {resultado['valor']} | "
              f"Peso: {resultado['peso']}/{capacidade} | "
              f"Freq: {resultado['frequencia']}")
    
    if invalidos_ordenados:
        print(f"\n=== SOLUÇÕES INVÁLIDAS (Top {top_k}) ===")
        for i, resultado in enumerate(invalidos_ordenados[:top_k]):
            print(f"{i+1}. {resultado['bitstring']} | "
                  f"Valor: {resultado['valor']} | "
                  f"Peso: {resultado['peso']}/{capacidade} | "
                  f"EXCEDE: {resultado['peso'] - capacidade}")
    
    return validos_ordenados, invalidos_ordenados

In [ ]:
def custo_esperado_otimizado(params, qc, gama, beta, profundidade, backend, 
                           valor_itens, peso_itens, capacidade, shots=1000):
    """
    Versão otimizada do cálculo do custo esperado
    """
    # Prepara o circuito com os parâmetros
    params_dict = {gama[i]: params[i] for i in range(profundidade)}
    params_dict.update({beta[i]: params[profundidade + i] for i in range(profundidade)})
    
    assigned_qc = qc.assign_parameters(params_dict)
    transpiled_qc = transpile(assigned_qc, backend)
    
    # Executa
    job = backend.run(transpiled_qc, shots=shots)
    result = job.result()
    counts = result.get_counts()
    
    # Calcula custo total de forma otimizada
    custo_total = 0
    n = len(valor_itens)
    
    for bitstring, freq in counts.items():
        # Avalia a bitstring
        resultado = avalia_bitstring_melhorada(bitstring, valor_itens, peso_itens, capacidade)
        
        # Usa o valor (negativo pois queremos maximizar)
        # Em QUBO, minimizamos, então convertemos maximização para minimização
        valor = resultado['valor']
        custo_bitstring = -valor  # Negativo porque queremos maximizar o valor
        
        # Penalidade adicional para soluções inválidas
        if not resultado['valido']:
            custo_bitstring += 1000  # Penalidade grande
        
        custo_total += custo_bitstring * (freq / shots)
    
    return custo_total

def funcao_objetivo_wrapper(params, qc, gama, beta, profundidade, backend,
                          valor_itens, peso_itens, capacidade, shots=1000):
    """
    Wrapper para otimizadores que sempre minimizam
    """
    return custo_esperado_otimizado(params, qc, gama, beta, profundidade, backend,
                                  valor_itens, peso_itens, capacidade, shots)

In [ ]:
from qiskit_aer import AerSimulator
from scipy.optimize import minimize
import numpy as np
from qiskit.visualization import plot_histogram

# Backend correto (AerSimulator em vez de BasicSimulator)
backend = AerSimulator()
shots = 10000

# Otimização com função corrigida
initial_params = np.random.uniform(0, np.pi, 2 * profundidade)

result = minimize(
    fun=custo_esperado_otimizado(
        params, qc, gama, beta, profundidade, backend,
        valor_itens, peso_itens, capacidade, shots=1000  # Menos shots durante otimização
    ),
    x0=initial_params,
    method="COBYLA",
    options={"maxiter": 200}
)

# Melhores parâmetros
optimal_params = result.x
print("Parâmetros otimizados (γ, β):", optimal_params)
print(f"Otimização {'bem-sucedida' if result.success else 'falhou'}")
print(f"Melhor valor da função: {result.fun}")

# Executar com parâmetros otimizados (mais shots para resultado final)
params_dict = {gama[i]: optimal_params[i] for i in range(profundidade)}
params_dict.update({beta[i]: optimal_params[profundidade + i] for i in range(profundidade)})
assigned_qc = qc.assign_parameters(params_dict)
transpiled_qc = transpile(assigned_qc, backend)
job = backend.run(transpiled_qc, shots=shots)
result = job.result()
counts = result.get_counts()

print("\nContagens de bitstrings (otimizadas):")
print(counts)

# Visualizar distribuição
plot_histogram(counts)

# Pós-processar resultados com função melhorada
print("\n" + "="*60)
print("ANÁLISE DETALHADA DOS RESULTADOS")
print("="*60)

# Usar a função de análise que criamos
melhores_validos, melhores_invalidos = analisa_resultados_qaoa(
    counts, valor_itens, peso_itens, capacidade, top_k=10
)

# Encontrar a solução ótima clássica para comparação
from itertools import product

def solucao_otima_classica(valor_itens, peso_itens, capacidade):
    """Encontra a solução ótima via força bruta (para comparação)"""
    n = len(valor_itens)
    melhor_valor = 0
    melhor_combinacao = None
    
    for bits in product([0, 1], repeat=n):
        valor = sum(valor_itens[i] * bits[i] for i in range(n))
        peso = sum(peso_itens[i] * bits[i] for i in range(n))
        
        if peso <= capacidade and valor > melhor_valor:
            melhor_valor = valor
            melhor_combinacao = bits
    
    return ''.join(str(b) for b in melhor_combinacao), melhor_valor

# Calcular solução ótima
sol_otima, valor_otimo = solucao_otima_classica(valor_itens, peso_itens, capacidade)
print(f"\n🔍 SOLUÇÃO ÓTIMA CLÁSSICA: {sol_otima} | Valor: {valor_otimo}")

# Comparar com melhor solução do QAOA
if melhores_validos:
    melhor_qaoa = melhores_validos[0]
    print(f"🎯 MELHOR SOLUÇÃO QAOA: {melhor_qaoa['bitstring']} | Valor: {melhor_qaoa['valor']}")
    
    # Taxa de aproximação
    taxa_aproximacao = melhor_qaoa['valor'] / valor_otimo if valor_otimo > 0 else 1
    print(f"📊 TAXA DE APROXIMAÇÃO: {taxa_aproximacao:.2%}")
    
    # Estatísticas
    n_validas = len(melhores_validos)
    n_totais = len(counts)
    print(f"📈 SOLUÇÕES VÁLIDAS: {n_validas}/{n_totais} ({n_validas/n_totais:.1%})")
    
    # Frequência da melhor solução
    freq_melhor = melhor_qaoa['frequencia']
    print(f"📏 FREQUÊNCIA DA MELHOR: {freq_melhor} shots")
else:
    print("❌ Nenhuma solução válida encontrada pelo QAOA")

# Salvar resultados para relatório
resultados_experimento = {
    'parametros_otimos': optimal_params.tolist(),
    'melhor_solucao': melhores_validos[0] if melhores_validos else None,
    'solucao_otima': sol_otima,
    'valor_otimo': valor_otimo,
    'counts': counts,
    'taxa_aproximacao': taxa_aproximacao if melhores_validos else 0
}

print("\n✅ EXPERIMENTO CONCLUÍDO!")

In [ ]:
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit import Parameter
from qiskit_aer import AerSimulator
from scipy.optimize import minimize
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt
from itertools import product

# ============================================================================
# DADOS DO PROBLEMA K1 - COM NORMALIZAÇÃO
# ============================================================================

pesos = [95, 4, 60, 32, 23, 72, 80, 62, 65, 46]
valores = [55, 10, 47, 5, 4, 50, 8, 61, 85, 87]
capacidade = 269
profundidade = 3

print("=== PROBLEMA DA MOCHILA K1 ===")
print(f"Pesos: {pesos}")
print(f"Valores: {valores}")
print(f"Capacidade: {capacidade}")

# ============================================================================
# 1. NORMALIZAÇÃO DOS DADOS - CRÍTICO PARA EVITAR ESTOURO NUMÉRICO
# ============================================================================

def normaliza_dados(valores, pesos, capacidade):
    """Normaliza dados para evitar estouro numérico"""
    escala_valores = max(valores)
    escala_pesos = max(pesos)
    
    valores_norm = [v / escala_valores for v in valores]
    pesos_norm = [p / escala_pesos for p in pesos]
    capacidade_norm = capacidade / escala_pesos
    
    return valores_norm, pesos_norm, capacidade_norm, escala_valores, escala_pesos

valores_norm, pesos_norm, capacidade_norm, escala_valores, escala_pesos = normaliza_dados(valores, pesos, capacidade)

print(f"\nDados normalizados:")
print(f"Valores normalizados: {[f'{v:.3f}' for v in valores_norm]}")
print(f"Pesos normalizados: {[f'{p:.3f}' for p in pesos_norm]}")
print(f"Capacidade normalizada: {capacidade_norm:.3f}")

# ============================================================================
# 2. FORMULAÇÃO QUBO COM COEFICIENTES CONTROLADOS
# ============================================================================

def cria_qubo_controlada(valores, pesos, capacidade, penalidade):
    """QUBO com coeficientes controlados para evitar estouro"""
    n = len(valores)
    Q = np.zeros((n, n))
    
    for i in range(n):
        # Termo linear - valores entre ~0.1 e 1.0
        Q[i, i] = -valores[i] + penalidade * pesos[i]**2 - 2 * penalidade * capacidade * pesos[i]
        
    for i in range(n):
        for j in range(i + 1, n):
            Q[i, j] = 2 * penalidade * pesos[i] * pesos[j]
            
    return Q

# Penalidade MUCHO menor para dados normalizados
penalidade = 2.0  # Em vez de 100+
qubo_matrix = cria_qubo_controlada(valores_norm, pesos_norm, capacidade_norm, penalidade)

print(f"\nMatriz QUBO criada")
print(f"Maior coeficiente: {np.max(np.abs(qubo_matrix)):.3f}")
print(f"Faixa de coeficientes: [{np.min(qubo_matrix):.3f}, {np.max(qubo_matrix):.3f}]")

# ============================================================================
# 3. HAMILTONIANO E CIRCUITO - MESMA LÓGICA ANTERIOR
# ============================================================================

def cria_hamiltoniano_simples(qubo_matrix):
    n = len(qubo_matrix)
    pauli_list = []
    coeffs_list = []
    
    for i in range(n):
        coef = qubo_matrix[i, i] / 2
        for j in range(n):
            if j != i:
                coef += qubo_matrix[i, j] / 4
        
        pauli_str = ['I'] * n
        pauli_str[i] = 'Z'
        pauli_list.append(''.join(pauli_str))
        coeffs_list.append(float(coef))
    
    constante = sum(qubo_matrix[i, i] / 2 for i in range(n))
    for i in range(n):
        for j in range(i + 1, n):
            constante += qubo_matrix[i, j] / 4
    
    pauli_list.append('I' * n)
    coeffs_list.append(float(constante))
    
    return SparsePauliOp(pauli_list, coeffs_list)

hamiltoniano_c = cria_hamiltoniano_simples(qubo_matrix)

def cria_circuito_qaoa_estavel(n_qubits, profundidade):
    qc = QuantumCircuit(n_qubits)
    qc.h(range(n_qubits))
    
    gammas = [Parameter(f'γ_{i}') for i in range(profundidade)]
    betas = [Parameter(f'β_{i}') for i in range(profundidade)]
    
    for p in range(profundidade):
        for i in range(n_qubits):
            qc.rz(2 * gammas[p] * qubo_matrix[i, i], i)
        for i in range(n_qubits):
            qc.rx(2 * betas[p], i)
    
    qc.measure_all()
    return qc, gammas, betas

n_qubits = len(valores)
qc, gammas, betas = cria_circuito_qaoa_estavel(n_qubits, profundidade)

# ============================================================================
# 4. FUNÇÃO DE CUSTO COM NORMALIZAÇÃO E CONTROLE
# ============================================================================

def funcao_custo_controlada(params, qc, gammas, betas, backend, shots=1000):
    """Função de custo com controle de escala"""
    try:
        param_dict = {}
        for i in range(len(gammas)):
            param_dict[gammas[i]] = float(params[i])
            param_dict[betas[i]] = float(params[len(gammas) + i])
        
        qc_param = qc.assign_parameters(param_dict)
        qc_transpiled = transpile(qc_param, backend)
        
        job = backend.run(qc_transpiled, shots=shots)
        result = job.result()
        counts = result.get_counts()
        
        custo_total = 0.0
        for bitstring, count in counts.items():
            bits = [int(b) for b in bitstring[::-1]]
            
            # Calcula valor QUBO normalizado
            valor_qubo = 0.0
            for i in range(n_qubits):
                for j in range(n_qubits):
                    if i == j:
                        valor_qubo += qubo_matrix[i, j] * bits[i]
                    elif i < j:
                        valor_qubo += qubo_matrix[i, j] * bits[i] * bits[j]
            
            custo_total += valor_qubo * (count / shots)
        
        print(f"Parâmetros: {[f'{p:.3f}' for p in params]}, Custo: {custo_total:.3f}")
        return float(custo_total)
        
    except Exception as e:
        print(f"Erro: {e}")
        return 1000.0

# ============================================================================
# 5. SOLUÇÃO ÓTIMA CLÁSSICA (PARA DADOS NORMALIZADOS)
# ============================================================================

def solucao_otima_classica_normalizada(valores, pesos, capacidade):
    n = len(valores)
    melhor_valor = 0
    melhor_combinacao = [0] * n
    
    for bits in product([0, 1], repeat=n):
        valor = sum(valores[i] * bits[i] for i in range(n))
        peso = sum(pesos[i] * bits[i] for i in range(n))
        
        if peso <= capacidade and valor > melhor_valor:
            melhor_valor = valor
            melhor_combinacao = bits
    
    bitstring = ''.join(str(b) for b in melhor_combinacao)
    peso_total = sum(pesos[i] * melhor_combinacao[i] for i in range(n))
    
    # Converte de volta para valores originais
    valor_original = sum(valores[i] * melhor_combinacao[i] * escala_valores for i in range(n))
    peso_original = sum(pesos[i] * melhor_combinacao[i] * escala_pesos for i in range(n))
    
    return bitstring, valor_original, peso_original

solucao_otima, valor_otimo, peso_otimo = solucao_otima_classica_normalizada(
    valores_norm, pesos_norm, capacidade_norm
)

print(f"\n🔍 SOLUÇÃO ÓTIMA CLÁSSICA:")
print(f"   Bitstring: {solucao_otima}")
print(f"   Valor: {valor_otimo:.1f} (esperado: 295)")
print(f"   Peso: {peso_otimo:.1f}/{capacidade}")

# ============================================================================
# 6. EXECUÇÃO DO QAOA COM CONTROLE DE ESCALA
# ============================================================================

print("\n=== EXECUTANDO QAOA COM DADOS NORMALIZADOS ===")

backend = AerSimulator()
shots = 1000

# Parâmetros iniciais mais conservadores
params_iniciais = np.random.uniform(0, 0.5, 2 * profundidade)
print(f"Parâmetros iniciais: {[f'{p:.3f}' for p in params_iniciais]}")

print("Otimizando...")
resultado = minimize(
    fun=funcao_custo_controlada,
    x0=params_iniciais,
    args=(qc, gammas, betas, backend, shots),
    method='COBYLA',
    options={'maxiter': 50, 'disp': True}
)

print(f"Sucesso: {resultado.success}")
print(f"Melhor custo: {resultado.fun:.6f}")

# ============================================================================
# 7. ANÁLISE DOS RESULTADOS COM CONVERSÃO
# ============================================================================

def avalia_bitstring_com_normalizacao(bitstring, valores_norm, pesos_norm, capacidade_norm, escala_valores, escala_pesos):
    bits = [int(b) for b in bitstring[::-1]]
    
    # Calcula com dados normalizados
    valor_norm = sum(valores_norm[i] * bits[i] for i in range(len(bits)))
    peso_norm = sum(pesos_norm[i] * bits[i] for i in range(len(bits)))
    valido = peso_norm <= capacidade_norm
    
    # Converte para valores originais
    valor_original = valor_norm * escala_valores
    peso_original = peso_norm * escala_pesos
    
    return valor_original, peso_original, valido

if resultado.success:
    params_otimos = resultado.x
    print(f"\nParâmetros otimizados: {[f'{p:.3f}' for p in params_otimos]}")
    
    # Execução final
    shots_final = 5000
    param_dict = {}
    for i in range(len(gammas)):
        param_dict[gammas[i]] = float(params_otimos[i])
        param_dict[betas[i]] = float(params_otimos[len(gammas) + i])
    
    qc_final = qc.assign_parameters(param_dict)
    qc_final = transpile(qc_final, backend)
    
    print("Executando circuito final...")
    job_final = backend.run(qc_final, shots=shots_final)
    result_final = job_final.result()
    counts_final = result_final.get_counts()
    
    print(f"\n📊 RESULTADOS OBTIDOS:")
    print(f"Total de bitstrings únicas: {len(counts_final)}")
    
    # Analisar resultados
    resultados_validos = []
    for bitstring, freq in counts_final.items():
        valor, peso, valido = avalia_bitstring_com_normalizacao(
            bitstring, valores_norm, pesos_norm, capacidade_norm, escala_valores, escala_pesos
        )
        if valido:
            resultados_validos.append((bitstring, valor, peso, freq))
    
    # Ordenar por valor
    resultados_validos.sort(key=lambda x: x[1], reverse=True)
    
    print(f"\n🎯 TOP 10 SOLUÇÕES VÁLIDAS:")
    for i, (bitstring, valor, peso, freq) in enumerate(resultados_validos[:10]):
        print(f"{i+1:2d}. {bitstring} | Valor: {valor:6.1f} | Peso: {peso:6.1f}/{capacidade} | Freq: {freq:4d}")
    
    if resultados_validos:
        melhor_bitstring, melhor_valor, melhor_peso, melhor_freq = resultados_validos[0]
        taxa_aproximacao = melhor_valor / 295.0  # Valor ótimo conhecido
        
        print(f"\n🏆 MELHOR SOLUÇÃO QAOA:")
        print(f"   Bitstring: {melhor_bitstring}")
        print(f"   Valor: {melhor_valor:.1f} (Ótimo: 295)")
        print(f"   Peso: {melhor_peso:.1f}/{capacidade}")
        print(f"   Frequência: {melhor_freq} ({melhor_freq/shots_final:.1%})")
        print(f"   Taxa de aproximação: {taxa_aproximacao:.2%}")
        
        if abs(melhor_valor - 295) < 1:
            print("   🎉 SOLUÇÃO ÓTIMA ENCONTRADA!")
        elif taxa_aproximacao > 0.9:
            print("   💡 Solução de alta qualidade!")
    else:
        print("❌ Nenhuma solução válida encontrada")
    
    # Plotar histograma das top 20 soluções
    top_20 = dict(sorted(counts_final.items(), key=lambda x: x[1], reverse=True)[:20])
    plt.figure(figsize=(12, 6))
    plot_histogram(top_20)
    plt.title(f"QAOA - Problema K1 (p={profundidade}) - Top 20")
    plt.show()

else:
    print("❌ Otimização não convergiu. Analisando resultados com parâmetros atuais...")
    
    # Analisar mesmo sem convergência
    param_dict = {}
    for i in range(len(gammas)):
        param_dict[gammas[i]] = float(params_iniciais[i])
        param_dict[betas[i]] = float(params_iniciais[len(gammas) + i])
    
    qc_test = qc.assign_parameters(param_dict)
    qc_test = transpile(qc_test, backend)
    
    job_test = backend.run(qc_test, shots=1000)
    result_test = job_test.result()
    counts_test = result_test.get_counts()
    
    print("\nResultados com parâmetros iniciais:")
    for bitstring, freq in list(counts_test.items())[:10]:
        valor, peso, valido = avalia_bitstring_com_normalizacao(
            bitstring, valores_norm, pesos_norm, capacidade_norm, escala_valores, escala_pesos
        )
        status = "VÁLIDO" if valido else "INVÁLIDO"
        print(f"  {bitstring} | Freq: {freq} | Valor: {valor:.1f} | Peso: {peso:.1f} | {status}")

print("\n✅ ANÁLISE CONCLUÍDA!")

In [29]:
valor_itens = [44, 46, 90, 72, 91, 40, 75, 35, 8, 54, 78, 40, 77, 15, 61, 17, 75, 29, 75, 63]
peso_itens = [92, 4, 43, 83, 84, 68, 92, 82, 6, 44, 32, 18, 56, 83, 25, 96, 70, 48, 14, 58]
capacidade = 878
qtd_itens = len(valor_itens)
penalidade = max(valor_itens) * 10 

print(f"Capacidade: {capacidade}")
print(f"Peso: {peso_itens}, Quantidade: {qtd_itens}")
print(f"Valor: {valor_itens}, Quantidade: {qtd_itens}")
print(f"Penalidade: {penalidade}")
print(f"Total itens: {sum(valor_itens)}")
print(f"Total pesos: {sum(peso_itens)}")
if qtd_itens == len(peso_itens):
    print(f"OK: {qtd_itens}")
else:
    print("Lista itens diferente de lista pesos")       

Capacidade: 878
Peso: [92, 4, 43, 83, 84, 68, 92, 82, 6, 44, 32, 18, 56, 83, 25, 96, 70, 48, 14, 58], Quantidade: 20
Valor: [44, 46, 90, 72, 91, 40, 75, 35, 8, 54, 78, 40, 77, 15, 61, 17, 75, 29, 75, 63], Quantidade: 20
Penalidade: 910
Total itens: 1085
Total pesos: 1098
OK: 20


In [30]:
# (Dados da mensagem anterior: valor_itens, peso_itens, etc.)
def cria_qubo(valor_itens, peso_itens, capacidade, penalidade):
    n = len(valor_itens)
    qubo_matrix = np.zeros([n, n])
    
    # Primeiro, termos lineares e quadráticos no diag
    for i in range(n):
        qubo_matrix[i, i] = -valor_itens[i] + penalidade * (peso_itens[i]**2) - 2 * penalidade * peso_itens[i] * capacidade
    
    # Off-diag simétricos: +2 P w_i w_j pra i != j (mas só setamos i<j e copiamos)
    for i in range(n):
        for j in range(i + 1, n):
            q_ij = 2 * penalidade * peso_itens[i] * peso_itens[j]
            qubo_matrix[i, j] = q_ij
            qubo_matrix[j, i] = q_ij  # <- Isso corrige a simetria!
    
    # Sanity check
    if not np.allclose(qubo_matrix, qubo_matrix.T):
        print("AVISO: Matriz QUBO não simétrica!")
    
    # Teste rápido: custo pra x= todos 0 (deve ser 0)
    x_zero = np.zeros(n)
    custo_zero = np.dot(x_zero, np.dot(qubo_matrix, x_zero))
    print(f"Sanity: Custo pra x=0: {custo_zero} (deve ser ~0)")
    
    return qubo_matrix

qubo_matrix = cria_qubo(valor_itens, peso_itens, capacidade, penalidade)
print("Matriz QUBO (primeiras 5x5, corrigida):")
print(qubo_matrix[:5, :5])  # Mais legível que full 20x20

Sanity: Custo pra x=0: 0.0 (deve ser ~0)
Matriz QUBO (primeiras 5x5, corrigida):
[[-1.39310124e+08  6.69760000e+05  7.19992000e+06  1.38975200e+07
   1.40649600e+07]
 [ 6.69760000e+05 -6.37732600e+06  3.13040000e+05  6.04240000e+05
   6.11520000e+05]
 [ 7.19992000e+06  3.13040000e+05 -6.70297800e+07  6.49558000e+06
   6.57384000e+06]
 [ 1.38975200e+07  6.04240000e+05  6.49558000e+06 -1.26361762e+08
   1.26890400e+07]
 [ 1.40649600e+07  6.11520000e+05  6.57384000e+06  1.26890400e+07
  -1.27807771e+08]]


In [7]:
# (Assumindo qubo_matrix do anterior)
def hamiltoniano_de_custo(qubo_matrix):  # Nome corrigido
    n = len(qubo_matrix)
    h_custo = SparsePauliOp.from_list([], num_qubits=n)  # Inicia vazio (melhor que I com 0)
    
    # Lineares completos: h_k = -0.5 * sum_j Q_{k j} (full row sum)
    h = -0.5 * np.sum(qubo_matrix, axis=1)  # Vetor de h_k
    for k in range(n):
        if abs(h[k]) > 1e-10:  # Evita zeros numéricos
            pauli_str = ["I"] * n
            pauli_str[k] = "Z"
            h_custo += SparsePauliOp("".join(pauli_str), coeffs=[h[k]])
    
    # Quadráticos: só i < j, J_ij = Q_ij / 4
    for i in range(n):
        for j in range(i + 1, n):
            if abs(qubo_matrix[i, j]) > 1e-10:
                pauli_str = ["I"] * n
                pauli_str[i] = "Z"
                pauli_str[j] = "Z"
                coef = qubo_matrix[i, j] / 4
                h_custo += SparsePauliOp("".join(pauli_str), coeffs=[coef])
    
    # Sanity check simples: teste com x = [1, 0, ..., 0]
    x_test = np.zeros(n)
    x_test[0] = 1.0
    e_qubo = np.dot(x_test, np.dot(qubo_matrix, x_test))
    # Pra Ising: Z_0 = -1 (x=1), outros Z=1 (x=0), mas só linear h_0 * (-1)
    e_ising_approx = h[0] * (-1) + sum(h[k] for k in range(1,n))  # Aproximação sem full sim
    print(f"Sanity: E_QUBO (x=[1,0,...]): {e_qubo:.2f}")
    print(f"E_Ising approx: {e_ising_approx:.2f} (deve bater dentro de ~1e-10)")
    
    return h_custo

hamiltoniano_c = hamiltoniano_de_custo(qubo_matrix)
print("Hamiltoniano de custo (corrigido):")
print(hamiltoniano_c)  # Mostra os termos principais

Sanity: E_QUBO (x=[1,0,...]): -139310124.00
E_Ising approx: -155178161.50 (deve bater dentro de ~1e-10)
Hamiltoniano de custo (corrigido):
SparsePauliOp(['IIIIIIIIIIIIIIIIIIII', 'ZIIIIIIIIIIIIIIIIIII', 'IZIIIIIIIIIIIIIIIIII', 'IIZIIIIIIIIIIIIIIIII', 'IIIZIIIIIIIIIIIIIIII', 'IIIIZIIIIIIIIIIIIIII', 'IIIIIZIIIIIIIIIIIIII', 'IIIIIIZIIIIIIIIIIIII', 'IIIIIIIZIIIIIIIIIIII', 'IIIIIIIIZIIIIIIIIIII', 'IIIIIIIIIZIIIIIIIIII', 'IIIIIIIIIIZIIIIIIIII', 'IIIIIIIIIIIZIIIIIIII', 'IIIIIIIIIIIIZIIIIIII', 'IIIIIIIIIIIIIZIIIIII', 'IIIIIIIIIIIIIIZIIIII', 'IIIIIIIIIIIIIIIZIIII', 'IIIIIIIIIIIIIIIIZIII', 'IIIIIIIIIIIIIIIIIZII', 'IIIIIIIIIIIIIIIIIIZI', 'IIIIIIIIIIIIIIIIIIIZ', 'ZZIIIIIIIIIIIIIIIIII', 'ZIZIIIIIIIIIIIIIIIII', 'ZIIZIIIIIIIIIIIIIIII', 'ZIIIZIIIIIIIIIIIIIII', 'ZIIIIZIIIIIIIIIIIIII', 'ZIIIIIZIIIIIIIIIIIII', 'ZIIIIIIZIIIIIIIIIIII', 'ZIIIIIIIZIIIIIIIIIII', 'ZIIIIIIIIZIIIIIIIIII', 'ZIIIIIIIIIZIIIIIIIII', 'ZIIIIIIIIIIZIIIIIIII', 'ZIIIIIIIIIIIZIIIIIII', 'ZIIIIIIIIIIIIZIIIIII', 'ZIIIIIIIIIIIIIZIIIII', 'ZIIII

In [13]:


# (Assumindo valor_itens, peso_itens do topo)
def hamiltoniano_de_mixer(valor_itens, peso_itens, uniforme=False, normalize_sum=False):
    n = len(valor_itens)
    # Fix: Init vazio com num_qubits=n
    h_mix = SparsePauliOp.from_list([], num_qubits=n)
    
    if uniforme:
        coefs = [1.0] * n
        print("Usando mixer uniforme (todos coef=1.0)")
    else:
        razao = []
        for i in range(n):
            if peso_itens[i] == 0:
                razao.append(0.0)
                print(f"Aviso: peso[i={i}]=0, razao=0")
            else:
                razao.append(valor_itens[i] / peso_itens[i])
        
        # Fix sintaxe: max primeiro, check separado
        max_razao = max(razao) if razao else 0.0
        if max_razao == 0:
            max_razao = 1.0  # Fallback pra evitar div/0 total
            print("Aviso: Todas razao=0; usando uniforme implícito")
            coefs = [1.0] * n
        else:
            coefs = [r / max_razao for r in razao]
            if normalize_sum:
                sum_razao = sum(razao)
                if sum_razao > 0:
                    coefs = [r / sum_razao * n for r in razao]  # Soma exata = n
                    print("Normalizado por soma (total força = n)")
        
        # Debug: top 3 itens por razao
        top_idx = np.argsort(razao)[-3:][::-1]
        print(f"Top 3 itens por v/w: idx {top_idx}, razoes {[razao[i] for i in top_idx]}")
    
    for i in range(n):
        pauli_str = ["I"] * n
        pauli_str[i] = "X"
        h_mix += SparsePauliOp("".join(pauli_str), coeffs=[coefs[i]])
    
    # Sanity: soma dos coefs
    print(f"Soma dos coefs mixer: {sum(coefs):.2f} (ideal ~{n} pra equilíbrio)")
    
    return h_mix

# Chame com heurística (default) ou uniforme=True; teste normalize_sum=True se quiser soma exata
hamiltoniano_m = hamiltoniano_de_mixer(valor_itens, peso_itens, uniforme=False, normalize_sum=False)
print("Hamiltoniano de mistura (corrigido e fixado):")
print(hamiltoniano_m)

Top 3 itens por v/w: idx [ 1 18 14], razoes [11.5, 5.357142857142857, 2.44]
Soma dos coefs mixer: 3.25 (ideal ~20 pra equilíbrio)
Hamiltoniano de mistura (corrigido e fixado):
SparsePauliOp(['IIIIIIIIIIIIIIIIIIII', 'XIIIIIIIIIIIIIIIIIII', 'IXIIIIIIIIIIIIIIIIII', 'IIXIIIIIIIIIIIIIIIII', 'IIIXIIIIIIIIIIIIIIII', 'IIIIXIIIIIIIIIIIIIII', 'IIIIIXIIIIIIIIIIIIII', 'IIIIIIXIIIIIIIIIIIII', 'IIIIIIIXIIIIIIIIIIII', 'IIIIIIIIXIIIIIIIIIII', 'IIIIIIIIIXIIIIIIIIII', 'IIIIIIIIIIXIIIIIIIII', 'IIIIIIIIIIIXIIIIIIII', 'IIIIIIIIIIIIXIIIIIII', 'IIIIIIIIIIIIIXIIIIII', 'IIIIIIIIIIIIIIXIIIII', 'IIIIIIIIIIIIIIIXIIII', 'IIIIIIIIIIIIIIIIXIII', 'IIIIIIIIIIIIIIIIIXII', 'IIIIIIIIIIIIIIIIIIXI', 'IIIIIIIIIIIIIIIIIIIX'],
              coeffs=[0.        +0.j, 0.0415879 +0.j, 1.        +0.j, 0.18200202+0.j,
 0.07543216+0.j, 0.0942029 +0.j, 0.0511509 +0.j, 0.07088847+0.j,
 0.03711559+0.j, 0.11594203+0.j, 0.10671937+0.j, 0.21195652+0.j,
 0.19323671+0.j, 0.11956522+0.j, 0.01571503+0.j, 0.21217391+0.j,
 0.01539855+0.j, 0.0931

In [14]:


# (Assumindo hamiltoniano_c, hamiltoniano_m do anterior)
def cria_qaoa(hamiltoniano_c, hamiltoniano_m, profundidade, draw_output="mpl"):
    n_c = hamiltoniano_c.num_qubits
    n_m = hamiltoniano_m.num_qubits
    if n_c != n_m:
        raise ValueError(f"Mismatch qubits: custo={n_c}, mixer={n_m}. Deve ser igual!")
    if profundidade < 1:
        raise ValueError("Profundidade deve ser >=1 para QAOA.")
    
    n = n_c
    qc = QuantumCircuit(n)
    
    # Estado inicial: sobreposição uniforme
    qc.h(range(n))
    
    # Parâmetros (convenção: "gamma" em vez de "gama")
    gamma = ParameterVector("gamma", profundidade)
    beta = ParameterVector("beta", profundidade)
    
    # Camadas QAOA: alterna custo -> mixer
    for i in range(profundidade):
        evo_custo = PauliEvolutionGate(hamiltoniano_c, time=gamma[i])
        qc.append(evo_custo, range(n))
        evo_mixer = PauliEvolutionGate(hamiltoniano_m, time=beta[i])
        qc.append(evo_mixer, range(n))
    
    # Medição em Z-basis
    qc.measure_all()
    
    # Debug: métricas do circuito
    print(f"QAOA criado: n={n} qubits, p={profundidade} camadas, depth={qc.depth()}, size={qc.size()}")
    
    # Draw flexível
    try:
        if draw_output == "mpl":
            qc.draw("mpl")
        else:
            print(qc.draw("text"))  # Fallback textual
    except ImportError:
        print("Aviso: mpl não disponível; use draw_output='text'.")
        print(qc.draw("text"))
    
    return qc, gamma, beta  # Note: gamma agora

# Use p=2 pra debug rápido; 4 pra full
profundidade = 2  # Sugestão: comece baixo
qc, gamma, beta = cria_qaoa(hamiltoniano_c, hamiltoniano_m, profundidade, draw_output="mpl")
print("Circuito QAOA criado com sucesso!")
# qc.draw("mpl")  # Já chamado na func

QAOA criado: n=20 qubits, p=2 camadas, depth=6, size=44
Circuito QAOA criado com sucesso!


In [ ]:
# (Assumindo valor_itens, peso_itens, capacidade do topo)
def avalia_bitstring(bitstring, valor_itens, peso_itens, capacidade, verbose=False):
    n = len(valor_itens)
    if not isinstance(bitstring, str) or len(bitstring) != n:
        if len(bitstring) < n:
            bitstring = bitstring.ljust(n, '0')  # Pad com 0 se curto
        else:
            raise ValueError(f"Bitstring '{bitstring}' deve ter len={n}; tem {len(bitstring)}")
    
    try:
        bits = np.array([int(b) for b in bitstring[::-1]])  # Vetorizado, little-endian fix
    except ValueError as e:
        raise ValueError(f"Bitstring inválida (não binária): {e}")
    
    valor_total = np.dot(bits, np.array(valor_itens))
    peso_total = np.dot(bits, np.array(peso_itens))
    valido = peso_total <= capacidade
    
    if verbose:
        print(f"Bitstring '{bitstring}' -> x={bits.tolist()[:5]}..., Valor={valor_total}, Peso={peso_total}, Válido={valido}")
    
    return valor_total, peso_total, valido

# Exemplo de teste (rode pra validar)
test_bitstring = '101000'[:6]  # Truncado pra demo; ajuste pra n=20
v_test, p_test = valor_itens[:6], peso_itens[:6]
cap_test = 100
print("Teste:")
avalia_bitstring('101000', v_test, p_test, cap_test, verbose=True)  # Ex: (44+72=116, 92+83=175 >100, False)

Teste:
Bitstring '101000' -> x=[0, 0, 0, 1, 0]..., Valor=112, Peso=151, Válido=False


(np.int64(112), np.int64(151), np.False_)

In [ ]:


# (Assumindo qc, gamma, beta, qubo_matrix do anterior)
def custo_esperado(params, qc, gamma, beta, profundidade, backend, qubo_matrix, shots=2048, verbose=False):
    n = profundidade * 2
    if len(params) != n:
        raise ValueError(f"Params deve ter {n} elementos; tem {len(params)}")
    
    # Binding params (consistente com "gamma")
    params_dict = {gamma[i]: params[i] for i in range(profundidade)}
    params_dict.update({beta[i]: params[profundidade + i] for i in range(profundidade)})
    
    assigned_qc = qc.assign_parameters(params_dict)
    transpiled_qc = transpile(assigned_qc, backend, optimization_level=3)  # Otimiza gates
    
    try:
        job = backend.run(transpiled_qc, shots=shots)
        result = job.result()
        counts = result.get_counts()
    except Exception as e:
        print(f"Erro no backend run: {e}")
        return float('inf')  # Penaliza params ruins
    
    if not counts:
        if verbose:
            print("Aviso: Counts vazio; retornando inf")
        return float('inf')
    
    custo_total = 0.0
    total_shots = sum(counts.values())
    for bitstring, freq in counts.items():
        bits_list = [int(b) for b in bitstring[::-1]]
        if len(bits_list) != qubo_matrix.shape[0]:
            continue  
        bits = np.array(bits_list)
        custo = np.dot(bits, np.dot(qubo_matrix, bits))  # <- Full, vetorizado!
        custo_total += custo * (freq / total_shots)
    
    if verbose:
        print(f"<H_C> estimado: {custo_total:.2f} (shots={total_shots}, unique={len(counts)})")
    
    return custo_total

In [ ]:
# Recrie o circuito com p=4 consistente (rode isso antes do teste!)
profundidade = 4  # Fix: Use o mesmo valor aqui e no teste
qc, gamma, beta = cria_qaoa(hamiltoniano_c, hamiltoniano_m, profundidade)

# Agora o teste corrigido
backend = BasicSimulator()
shots = 100  # Baixo pra debug rápido
initial_params = np.random.uniform(0, np.pi, 2 * profundidade)  # 8 pra p=4

# Chame com args corretos (qubo_matrix incluso, como corrigido)
custo_test = custo_esperado(initial_params, qc, gamma, beta, profundidade, backend, qubo_matrix, shots=shots, verbose=True)
print(f"Custo teste: {custo_test:.2f}")

# Opcional: Rode com params zero pra baseline (uniform superposition)
zero_params = np.zeros(2 * profundidade)
custo_zero = custo_esperado(zero_params, qc, gamma, beta, profundidade, backend, qubo_matrix, shots=shots, verbose=True)
print(f"Custo baseline (γ=β=0, uniform): {custo_zero:.2f} (deve ser ~média QUBO, negativo médio)")

QAOA criado: n=20 qubits, p=4 camadas, depth=10, size=48
<H_C> estimado: -328947878.45 (shots=100, unique=100)
Custo teste: -328947878.45
<H_C> estimado: -327467219.61 (shots=100, unique=100)
Custo baseline (γ=β=0, uniform): -327467219.61 (deve ser ~média QUBO, negativo médio)


In [32]:
    

# (Assumindo qc, gamma, beta, qubo_matrix, backend, profundidade=4, valor_itens etc. do anterior)
shots = 2048  # Balance: preciso mas rápido; suba pra 10000 local
initial_params = np.random.uniform(0, np.pi, 2 * profundidade)

# Minimize com lambda pra passar qubo_matrix
result = minimize(
    fun=lambda p: custo_esperado(p, qc, gamma, beta, profundidade, backend, qubo_matrix, shots),
    x0=initial_params,
    method="COBYLA",
    options={"maxiter": 300, "tol": 1e-3, "disp": False}  # Mais iters, tol pra convergência
)

optimal_params = result.x
print("Parâmetros otimizados (γ, β):", optimal_params)
print("Custo mínimo <H_C>:", result.fun)  # Deve ser negativo ~ -1000 pra bom run

# Run final otimizado
params_dict = {gamma[i]: optimal_params[i] for i in range(profundidade)}
params_dict.update({beta[i]: optimal_params[profundidade + i] for i in range(profundidade)})
assigned_qc = qc.assign_parameters(params_dict)
transpiled_qc = transpile(assigned_qc, backend, optimization_level=3, coupling_map=None)
job = backend.run(transpiled_qc, shots=shots)
result_final = job.result()
counts = result_final.get_counts()

print("Contagens de bitstrings (otimizadas, top 5):")
print(dict(sorted(counts.items(), key=lambda x: x[1], reverse=True)[:5]))


top_results = sorted(counts.items(), key=lambda x: x[1], reverse=True)[:10]
print("Top 10 soluções (otimizadas):")
best_valor = -float('inf')
best_bitstring = None
num_validos = 0
total_freq_validos = 0
for bitstring, freq in top_results:
    valor, peso, valido = avalia_bitstring(bitstring, valor_itens, peso_itens, capacidade)
    print(f"Bitstring: {bitstring}, Frequência: {freq}, Valor: {valor}, Peso: {peso}, Válido: {valido}")
    if valido:
        num_validos += 1
        total_freq_validos += freq


for bitstring, freq in counts.items():
    valor, peso, valido = avalia_bitstring(bitstring, valor_itens, peso_itens, capacidade)
    if valido and valor > best_valor:
        best_valor = valor
        best_bitstring = bitstring
    if valido:
        num_validos += 1  
        total_freq_validos += freq

pct_validos = (total_freq_validos / shots) * 100 if shots > 0 else 0
print(f"% Amostras válidas: {pct_validos:.1f}% ({num_validos} unique válidas)")

if best_bitstring is None:
    print("Nenhuma solução válida encontrada. Aumente penalidade!")
else:
    print(f"\nMelhor solução válida: Bitstring {best_bitstring}, Valor {best_valor}")
    # Opcional: detalhes
    _, peso_best, _ = avalia_bitstring(best_bitstring, valor_itens, peso_itens, capacidade)
    print(f"Detalhes: Peso {peso_best} <= {capacidade}")

KeyboardInterrupt: 

In [25]:
try:
    plot_histogram(counts)
    print("Histograma plotado!")
except ImportError:
    print("Aviso: plot_histogram falhou; instale matplotlib.")

Histograma plotado!


In [ ]:
def knapsack_dp(values, weights, capacity):
    n = len(values)
    #taabela DP: dp[i][w] 
    dp = [[0 for _ in range(capacity + 1)] for _ in range(n + 1)]
    
    for i in range(1, n + 1):
        for w in range(capacity + 1):
            if weights[i-1] <= w:
                # Max 
                dp[i][w] = max(dp[i-1][w], dp[i-1][w - weights[i-1]] + values[i-1])
            else:
                dp[i][w] = dp[i-1][w]
    
    max_value = dp[n][capacity]
    
    # Backtrack
    bitstring = ['0'] * n
    w = capacity
    for i in range(n, 0, -1):
        if weights[i-1] <= w and dp[i][w] == dp[i-1][w - weights[i-1]] + values[i-1]:
            bitstring[i-1] = '1' #inclui o item
            w -= weights[i-1] #atualiza capacidade
    
    peso_total = sum(weights[i] for i in range(n) if bitstring[i] == '1')
    return max_value, ''.join(bitstring), peso_total

max_val, bitstr_otima, peso_otimo = knapsack_dp(valor_itens, peso_itens, capacidade)
print(f"Ótimo exato (DP): {max_val}")
print(f"Bitstring ótima: {bitstr_otima}")
print(f"Peso total ótimo: {peso_otimo} (<= {capacidade})")

Ótimo exato (DP): 1024
Bitstring ótima: 11111111111110101011
Peso total ótimo: 871 (<= 878)


In [ ]:
print()
print("Gap entre DP e QAOA(Hibrido)\n") 